# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [4]:
!python3 -m pip install -q langchain-elasticsearch

In [9]:
%pip install -qU langchain-elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [10]:
!python3 -m pip install -qU jq lark langchain langchain_openai tiktoken



In [11]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [13]:
# Retrieve API keys securely
ELASTIC_CLOUD_ID = getpass("Enter your Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Enter your Elastic API Key: ")
OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Connect to Elasticsearch using ElasticsearchStore
vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,  # Elastic Cloud ID
    es_api_key=ELASTIC_API_KEY,    # Elastic API key
    index_name="reviews_sentiment_cluster_nlp",  # Name of the index where data will be stored
    embedding=embeddings  # Use OpenAI embeddings for text
)

# Optionally, print to confirm that the vector store has been initialized
print("Elasticsearch connection established and vector store created!")

Elasticsearch connection established and vector store created!


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [14]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [16]:
""" pip install jq """

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define a function to populate metadata
def metadata_func(record: dict, metadata: dict) -> dict:
    # Example: Populate metadata fields
    metadata["name"] = record.get("name", "")
    metadata["summary"] = record.get("summary", "")
    metadata["url"] = record.get("url", "")
    metadata["category"] = record.get("category", "")
    metadata["updated_at"] = record.get("updated_at", "")
    return metadata

# Load documents from JSON file
loader = JSONLoader(
    file_path="temp.json",  # Path to your JSON file
    jq_schema=".[]",  # jq schema to extract records
    content_key="content",  # Key in JSON for the main content
    metadata_func=metadata_func  # Function to populate metadata
)

# Split documents into 800 token passages with 400 token overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800,  # Chunk size: 800 tokens
    chunk_overlap=400  # Overlap: 400 tokens between chunks
)

# Load and split the documents
docs = loader.load_and_split(text_splitter=text_splitter)

# Check the number of documents created and verify the chunks
print(f"Number of document chunks created: {len(docs)}")


Number of document chunks created: 15


### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [23]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import ElasticsearchStore
from langchain_openai.embeddings import OpenAIEmbeddings
from getpass import getpass
from elasticsearch.helpers import BulkIndexError

# Step 1: Load the JSON file and metadata
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["name"] = record.get("name", "")
    metadata["summary"] = record.get("summary", "")
    metadata["url"] = record.get("url", "")
    metadata["category"] = record.get("category", "")
    metadata["updated_at"] = record.get("updated_at", "")
    return metadata

loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func
)

# Step 2: Split documents into passages
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800,
    chunk_overlap=400
)

# Load and split documents
docs = loader.load_and_split(text_splitter=text_splitter)

# Step 3: Set up Elasticsearch connection and embedding model
ELASTIC_CLOUD_ID = getpass("Enter your Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Enter your Elasticsearch API Key: ")
OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Step 4: Index the documents into Elasticsearch with error handling
try:
    vectorstore = ElasticsearchStore.from_documents(
        docs,  # Pass the documents list (split passages) here
        embedding=embeddings,  # Ensure your embedding model is initialized
        es_cloud_id=ELASTIC_CLOUD_ID,
        es_api_key=ELASTIC_API_KEY,
        index_name="reviews_sentiment_cluster_nlp"
    )
    print("Documents indexed successfully!")

except BulkIndexError as e:
    # Catch bulk indexing errors and log details
    print(f"BulkIndexError: {e}")
    failed_docs = e.errors
    print(f"{len(failed_docs)} documents failed to index.")

except Exception as ex:
    # Catch any other exceptions
    print(f"An error occurred: {ex}")



Error adding texts: 10 document(s) failed to index.
First error reason: [1:4589] failed to parse field [metadata.updated_at] of type [date] in document with id 'dcc327cf-58c2-4cdd-9f0c-2db7552c4309'. Preview of field's value: ''


BulkIndexError: 10 document(s) failed to index.
10 documents failed to index.


# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [47]:
from langchain.retrievers import MultiQueryRetriever
from langchain.vectorstores import ElasticsearchStore
from langchain_openai.embeddings import OpenAIEmbeddings
from getpass import getpass

# Initialize embeddings
OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Set up Elasticsearch connection
ELASTIC_CLOUD_ID = getpass("Enter your Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Enter your Elasticsearch API Key: ")

# Initialize ElasticsearchStore to connect to your Elasticsearch index
vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="reviews_sentiment_cluster_nlp",  # Ensure you use the correct index name
    embedding=embeddings
)

# Initialize the MultiQueryRetriever
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Now you can use 'retriever' in your context or question-answering pipeline


In [48]:
# Import the updated ElasticsearchStore from langchain_elasticsearch
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.embeddings import OpenAIEmbeddings
from getpass import getpass

# Step 3: Set up Elasticsearch connection and embedding model
ELASTIC_CLOUD_ID = getpass("Enter your Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Enter your Elasticsearch API Key: ")
OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Use ElasticsearchStore from langchain_elasticsearch package
vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="reviews_sentiment_cluster_nlp",  # Ensure you use the correct index name
    embedding=embeddings
)

print("Documents indexed successfully!")


KeyboardInterrupt: Interrupted by user

In [50]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document
from langchain.vectorstores import ElasticsearchStore
from langchain_openai.embeddings import OpenAIEmbeddings
from getpass import getpass
import logging

# Set up logging to monitor retriever activities
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Step 3: Set up Elasticsearch connection and embedding model
ELASTIC_CLOUD_ID = getpass("Enter your Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Enter your Elasticsearch API Key: ")
OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Initialize ElasticsearchStore
vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="reviews_sentiment_cluster_nlp",  # Ensure you use the correct index name
    embedding=embeddings
)

# Initialize the MultiQueryRetriever
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)


# Define the LLM context prompt template
LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Be as verbose and educational in your response as possible.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

# Define the document prompt template for formatting documents
LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """---
    SOURCE: {name}
    {page_content}
    ---"""
)

# Function to combine documents into a formatted string
def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# Define the retriever and parallel processing of context and question
_context = RunnableParallel(
    context=retriever | _combine_documents,  # Fetch and combine documents
    question=RunnablePassthrough(),  # Pass the question as-is
)

# Create a chain that combines context, question, and LLM response
chain = _context | LLM_CONTEXT_PROMPT | llm

# Invoke the chain with a sample question
ans = chain.invoke("what is the nasa sales team?")

# Output the answer
print("---- Answer ----")
print(ans)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
I'm sorry, I don't know the answer to that question as it is not mentioned in the retrieved context. The context is about sales strategies and work policies for a tech company, not NASA.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

Iteration 1: Enhanced Prompting with User Feedback
In this iteration, we'll modify the prompt templates to gather user feedback after answering a question, which can help refine the model's understanding of what constitutes a good answer.

In [51]:
# Iteration 1: Enhanced Prompting with User Feedback

# Define an enhanced LLM context prompt template to include user feedback
LLM_CONTEXT_PROMPT_FEEDBACK = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Be as verbose and educational in your response as possible.

    context: {context}
    Question: "{question}"
    Answer:

    After answering, please ask the user for feedback: 
    "Was this answer helpful? (yes/no)"
    """
)

# Update the context definition to utilize the new prompt
_context_feedback = RunnableParallel(
    context=retriever | _combine_documents,  # Fetch and combine documents
    question=RunnablePassthrough(),  # Pass the question as-is
)

# Create a chain that combines context, question, and LLM response with feedback
chain_feedback = _context_feedback | LLM_CONTEXT_PROMPT_FEEDBACK | llm

# Invoke the chain with a sample question
ans_feedback = chain_feedback.invoke("What is the NASA sales team?")

# Output the answer with user feedback request
print("---- Answer ----")
print(ans_feedback)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----

I'm sorry, I don't know the answer to that question. Would you like me to try and find more information for you?


Iteration 2: Contextualization with Related Questions
In this iteration, we'll enhance the retriever to also provide related questions that could give the user more context or clarify their queries.

In [52]:
# Iteration 2: Contextualization with Related Questions

# Define an additional prompt template to include related questions
RELATED_QUESTIONS_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    Additionally, suggest related questions that the user might find useful.
    If you don't know the answer, just say that you don't know.

    context: {context}
    Question: "{question}"
    Answer:
    Related Questions:
    """
)

# Update the context definition to utilize the related questions prompt
_context_related = RunnableParallel(
    context=retriever | _combine_documents,  # Fetch and combine documents
    question=RunnablePassthrough(),  # Pass the question as-is
)

# Create a chain that combines context, question, and LLM response with related questions
chain_related = _context_related | RELATED_QUESTIONS_PROMPT | llm

# Invoke the chain with a sample question
ans_related = chain_related.invoke("What is the NASA sales team?")

# Output the answer along with related questions
print("---- Answer ----")
print(ans_related)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
1. What is the sales team structure at our company?
    2. How many members are on the sales team?
    3. What are the responsibilities of the sales team?
    4. How does the sales team contribute to the company's overall goals?
    5. What training and support is provided to the sales team?
    6. How is the sales team's performance evaluated?
    7. What strategies does the sales team use to reach their objectives?
    8. How does the sales team collaborate with other departments?
    9. What is the sales team's role in customer satisfaction?
    10. How does the sales team adapt to changes in the market?
